## Data Processing and cleaning

Functions to clean information on google colab

In [ ]:
# === 0) Setup: Mount Drive and Install Packages ===
from google.colab import drive
drive.mount('/content/drive')

!pip install openpyxl tqdm

import pandas as pd
import re
from pathlib import Path
from tqdm import tqdm

# === 1) Base directory and files ===
BASE_DIR = Path('/content/drive/MyDrive/nytimes/data_edited')
FILES = [
    'February-2025-Voter-Omnibus-Crosstabs-Edited.xlsx',
    'March-2025-Voter-Omnibus-Crosstabs-Edited.xlsx',
    'April-2025-Voter-Omnibus-Crosstabs-Edited.xlsx',
    'May-2025-Voter-Omnibus-Crosstabs-Edited.xlsx',
    'June-2025-Omnibus-Crosstabs-Edited.xlsx',
    'July-2025-Voter-Omnibus-Crosstabs-Edited.xlsx',
    'August-2025-Omnibus-Crosstabs-Edited.xlsx',
    'September-2025-Voter-Omnibus-Crosstabs-Edited.xlsx',
    'October-2025-Voter-Omnibus-Crosstabs-Edited.xlsx',
    'November-2025-Voter-Omnibus-Crosstabs-Edited.xlsx',
]

# === 2) Map filenames → YYYY-MM ===
month_map = {
    'February': '2025-02', 'March': '2025-03', 'April': '2025-04', 'May': '2025-05', 'June': '2025-06',
    'July': '2025-07', 'August': '2025-08', 'September': '2025-09', 'October': '2025-10', 'November': '2025-11'
}

def parse_month_from_filename(fn: str) -> str:
    for name, ym in month_map.items():
        if name in fn:
            return ym
    return 'unknown'

# === 3) Tidy function for one sheet ===
def tidy_one_sheet(df: pd.DataFrame, period: str, sheet_name: str, skipped_log: list):
    if df.shape[1] < 3:
        return pd.DataFrame()

    # Header rows: first two rows
    header_rows = df.iloc[0:2, :].copy()
    data = df.iloc[2:, :].dropna(how='all')  # Remove empty rows

    # Build MultiIndex for columns
    cols = []
    for j in range(df.shape[1]):
        if j == 0:
            cols.append(('__meta__', 'question'))
        elif j == 1:
            cols.append(('__meta__', 'response_option'))
        else:
            cat = str(header_rows.iat[0, j]).strip() if pd.notna(header_rows.iat[0, j]) else ''
            sub = str(header_rows.iat[1, j]).strip() if pd.notna(header_rows.iat[1, j]) else ''
            cols.append((cat, sub))

    data.columns = pd.MultiIndex.from_tuples(cols)

    # Extract meta columns
    meta = data['__meta__'].copy()
    meta.columns = ['question', 'response_option']

    # Drop rows where both question and response_option are NaN
    data = data.loc[~(meta['question'].isna() & meta['response_option'].isna())].copy()

    # Forward-fill question text
    qcol = meta['question'].ffill()

    # Split question into ID and text
    def split_q(x):
        if pd.isna(x): return (None, None)
        m = re.match(r"\s*\[(?P<qid>[^\]]+)\]\s*(?P<qtext>.*)$", str(x).strip())
        if m: return (m.group('qid'), m.group('qtext'))
        return (None, str(x))

    qid, qtext = zip(*qcol.map(split_q))

    # Stack demographic columns
    long_frames = []
    for (cat, sub) in data.columns:
        if cat == '__meta__':
            continue
        values = data[(cat, sub)].values
        if all(pd.isna(values)):  # If all values are NaN, log and skip
            skipped_log.append(f"Skipped block: Question '{qid[0]}' ({sheet_name}, {period}) for {cat}/{sub}")
            continue
        block = pd.DataFrame({
            'period': period,
            'sheet': sheet_name,
            'question_id': qid,
            'question_text': qtext,
            'response_option': meta['response_option'].values,
            'demo_category': cat,
            'demo_subcategory': sub,
            'value': values,
        })
        long_frames.append(block)

    out = pd.concat(long_frames, ignore_index=True) if long_frames else pd.DataFrame()

    # Convert to numeric
    def to_num(v):
        if pd.isna(v): return pd.NA
        s = str(v).strip()
        if s == '': return pd.NA
        s = s.replace(',', '')
        if s.endswith('%'):
            try: return float(s[:-1]) / 100.0
            except: return pd.NA
        try: return float(s)
        except: return pd.NA

    out['value_num'] = out['value'].map(to_num)
    return out

# === 4) Loop all files/sheets with progress bar ===
all_long, notes, skipped_log = [], [], []
for fn in tqdm(FILES, desc="Processing files"):
    fp = BASE_DIR / fn
    if not fp.exists():
        notes.append(f"Missing file {fn}")
        continue
    period = parse_month_from_filename(fn)
    try:
        xls = pd.ExcelFile(fp, engine='openpyxl')
        for sheet in xls.sheet_names:
            try:
                df = xls.parse(sheet_name=sheet, header=None)
                tidy = tidy_one_sheet(df, period, sheet, skipped_log)
                if not tidy.empty:
                    all_long.append(tidy)
            except Exception as e:
                notes.append(f"Error parsing sheet '{sheet}' in {fn}: {e}")
    except Exception as e:
        notes.append(f"Error opening {fn}: {e}")

# === 5) Concatenate & normalize response labels ===
if all_long:
    long_df = pd.concat(all_long, ignore_index=True)

    def norm_resp(s):
        if pd.isna(s): return s
        t = str(s).strip().lower()
        rep = {
            'rigth direction': 'right direction',
            'right direction': 'right direction',
            'wrong track': 'wrong track',
            'unsure': 'unsure',
            'dk': 'unsure',
            "don't know": 'unsure',
            'don’t know': 'unsure',
            'refused': 'unsure'
        }
        return rep.get(t, t)

    long_df['response_option_norm'] = long_df['response_option'].map(norm_resp)

    # Save outputs in Drive
    tidy_path = BASE_DIR / 'polls_2025_long_tidy.csv'
    codebook_path = BASE_DIR / 'polls_codebook_2025.csv'

    long_df.to_csv(tidy_path, index=False)
    codebook = (long_df.dropna(subset=['question_id', 'question_text'])
                        .drop_duplicates(['question_id','question_text'])
                        .sort_values('question_id'))
    codebook.to_csv(codebook_path, index=False)

    print(f"\n✅ Tidy data saved to: {tidy_path}")
    print(f"✅ Codebook saved to: {codebook_path}")
    print({
        'rows': len(long_df),
        'unique_questions': long_df['question_id'].nunique(),
        'periods': sorted(long_df['period'].dropna().unique().tolist())[:12],
        'demo_categories_examples': long_df['demo_category'].dropna().unique().tolist()[:10]
    })
else:
    print({'rows': 0, 'note': 'No rows produced; check file format and sheet structure.'})

print("\nNotes:")
print(notes)

print("\nSkipped blocks (empty data):")
for s in skipped_log[:20]:  # Show first 20
    print(s)
print(f"... Total skipped: {len(skipped_log)}")